In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

OSError: 'Seaborn' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [ ]:

import os
from urllib.request import urlretrieve

URL = 'https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD'

def get_fremont_data(filename='Fremont.csv', url=URL, forceDownload=False):
    if forceDownload or not os.path.exists(filename):
        urlretrieve(URL, 'Fremont.csv')
    data = pd.read_csv('Fremont.csv', index_col='Date', parse_dates=True)
    data.columns = ['West', 'East']
    data['Total'] = data['West'] + data['East']
    return data

In [ ]:
data = get_fremont_data()

In [ ]:
data.head()

In [ ]:
data.resample('W').sum().plot()

In [ ]:
data.resample('W').sum().plot()

In [ ]:
ax = data.resample('D').sum().rolling(365).sum().plot()
ax.set_ylim(0, None)

In [ ]:
data.groupby(data.index.time).mean().plot()

In [ ]:
pivoted = data.pivot_table('Total', index=data.index.time, columns=data.index.date)
pivoted.iloc[:5,:5]

In [ ]:
pivoted.plot(legend=False, alpha=0.01)